In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2)
tf.random.set_seed(2)

2024-01-06 18:52:25.897076: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-06 18:52:26.117451: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-06 18:52:26.861477: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-01-06 18:52:26.861575: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

# creating classifier

In [2]:
autoencoder_model = tf.keras.models.load_model(
    r"./saved_models/encoder_v3",
    custom_objects={"custom_loss": None},
)

autoencoder_model.trainable = False
model_input = autoencoder_model.layers[0].input
encoder_output = autoencoder_model.get_layer("encoder_output").output
encoder = keras.Model(inputs=[model_input], outputs=[encoder_output])

x = x_in = keras.layers.Input((256, 256, 3))

encoder_output = encoder(x)

x = keras.layers.Conv2D(
    8,
    kernel_size=3,
    strides=(2, 2),
    padding="same",
)(encoder_output)
x = keras.layers.Flatten()(x) #(encoder_output)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(5, "softmax")(x)

# create model
classifier = keras.Model(inputs=[x_in], outputs=[x])

classifier.summary()

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


2024-01-06 18:52:27.907915: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-06 18:52:27.945446: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2024-01-06 18:52:27.945470: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-01-06 18:52:27.945794: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 model (Functional)          (None, 16, 16, 16)        6840      
                                                                 
 conv2d (Conv2D)             (None, 8, 8, 8)           1160      
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 5)                 2565      
                                                                 
Total params: 10,565
Trainable params: 3,725
Non-trainable 

# load classifier

In [3]:
classifier = tf.keras.models.load_model(r"./saved_models/classifier_v2")
classifier.summary(expand_nested=True)

In [4]:
for l in classifier.layers:
    if isinstance(l, tf.keras.Model):
        for l2 in l.layers:
            print(l2)

    print(l.name)

input_1
model
conv2d
flatten
dropout
dense


# Save model weights as numpy file

the name of the file indicates the 'type' of the layer, the layer 'name', and 'activation', separeted by '.'.\
eg: Conv2D.encoder_output.Linear

In [5]:
def save_layer_file(id, layer, path):
    layer_name = layer.name
    layer_type = None
    activation_type = "Linear"
    weights, bias = [[], []]
    
    layer_config = layer.get_config()

    if isinstance(layer, tf.keras.layers.Conv2D):
        layer_type = "Conv2D"
        weights, bias = layer.get_weights()
        activation_type = layer_config["activation"]

    elif isinstance(layer, tf.keras.layers.DepthwiseConv2D):
        layer_type = "DepthwiseConv2D"
        weights, bias = layer.get_weights()
        activation_type = layer.get_config()["activation"]

    elif isinstance(layer, tf.keras.layers.Dense):
        layer_type = "Dense"
        weights, bias = layer.get_weights()
        activation_type = layer.get_config()["activation"]

    elif isinstance(layer, tf.keras.layers.Flatten):
        layer_type = "Flatten"

    elif isinstance(layer, tf.keras.layers.InputLayer):
        layer_type = "InputLayer"

    elif isinstance(layer, tf.keras.layers.Dropout):
        layer_type = "Dropout"

    else:
        print(f"layer type {type(layer)} not expected..")

    weight_path = f"{id}.{layer_type}.{layer_name}.{activation_type}.weights.npy"
    bias_path = f"{id}.{layer_type}.{layer_name}.{activation_type}.bias.npy"

    np.save(
        f"{path}/" + weight_path,
        weights,
    )
    np.save(
        f"{path}/" + bias_path,
        bias,
    )

    layerJson = {
        "layerName": layer_name,
        "layerType": layer_type,
        "activation": activation_type,
        "weightPath": weight_path,
        "biasPath": bias_path,
        "layer_config" : layer_config
    }

    return layerJson



save_path = f"./saved_models/classifier_v2"
modelJson = {
    "modelName": "classifier_v2",
    "layers": [],
}


index = 0
for l in classifier.layers:
    if isinstance(l, tf.keras.Model):
        for l2 in l.layers[1:]:
            layerJson = save_layer_file(index, l2, save_path)
            modelJson["layers"].append(layerJson)
            index += 1
    else:
        print(l.name)
        layerJson = save_layer_file(index, l, save_path)
        modelJson["layers"].append(layerJson)
        index += 1

import json

with open(f"{save_path}/model.json", "w") as file:
    file.write(json.dumps(modelJson))
    print("model exported ...")

input_1
conv2d
flatten
dropout
dense
model exported ...


# create the numpy dense/softmax layer

In [6]:
from Layers import Dense, Softmax

softmax = Softmax()
fc = Dense("dense", activation=softmax)
fc.load_weights(
    "/home/george/Desktop/dnn_processor/reference_algorithm/neural_network/saved_models/classifier_v2/7.Dense.dense.softmax.weights.npy",
    "/home/george/Desktop/dnn_processor/reference_algorithm/neural_network/saved_models/classifier_v2/7.Dense.dense.softmax.bias.npy",
)

# test dense/softmax layer

In [7]:
# the tensorflow layer and the numpy layer on the same input

denseLayer = classifier.get_layer("dense")

input_test = np.ones((32, denseLayer.get_weights()[0].shape[0]))

o_dut = fc(input_test, False).astype(np.float16).ravel()
o_dur = denseLayer(input_test).numpy().astype(np.float16).ravel()

np.savetxt("dut_fc.txt", o_dut)
np.savetxt("dur_fc.txt", o_dur)

assert o_dut.shape == o_dur.shape, f"shape test failed. { o_dut.shape} != {o_dur.shape}"
assert np.all(o_dut == o_dur), f"val test failed"

print("test_passed!")

test_passed!


# create conv2D numpy layer

In [8]:
from Layers import Conv2D, Tanh

conv2D_ref = classifier.get_layer("conv2d")
input_test = np.ones((1, 13, 13, conv2D_ref.weights[0].shape[-2]))


npConv2D = Conv2D(
    strides=conv2D_ref.get_config()["strides"],
    padding=conv2D_ref.get_config()["padding"],
    activation=None,
)
npConv2D.load_weights(
    weight_path="./saved_models/classifier_v2/4.Conv2D.conv2d.linear.weights.npy",
    bias_path="./saved_models/classifier_v2/4.Conv2D.conv2d.linear.bias.npy",
)

# test conv2D layer

In [9]:
o_dut = npConv2D(input_test, training=False).astype(np.float16).ravel()
o_dur = conv2D_ref(input_test).numpy().astype(np.float16).ravel()


np.savetxt("dut_conv.txt", o_dut)
np.savetxt("dur_conv.txt", o_dur)


assert o_dut.shape == o_dur.shape, f"shape test failed. { o_dut.shape} != {o_dur.shape}"
assert np.all(o_dut == o_dur), f"val test failed. Check dut_conv.txt and dur_conv.txt files "

print("test passed!")

test passed!


# create depthwise convolution

In [12]:
from Layers import DephtwiseConv2D, Tanh

depthwise_conv2d_ref = classifier.get_layer("model").get_layer("depthwise_conv2d")
input_test = np.ones((1, 13, 13, depthwise_conv2d_ref.weights[0].shape[-2]))

print(input_test.shape)

npDephtwiseConv2D = DephtwiseConv2D(
    strides=depthwise_conv2d_ref.get_config()["strides"],
    padding=depthwise_conv2d_ref.get_config()["padding"],
    activation=None,
)
npDephtwiseConv2D.load_weights(
    weight_path="./saved_models/classifier_v2/1.DepthwiseConv2D.depthwise_conv2d.linear.weights.npy",
    bias_path="./saved_models/classifier_v2/1.DepthwiseConv2D.depthwise_conv2d.linear.bias.npy",
)

(1, 13, 13, 3)


# test depthwise convolution

In [11]:
o_dut = npConv2D(input_test, training=False).astype(np.float16).ravel()
o_dur = conv2D_ref(input_test).numpy().astype(np.float16).ravel()


np.savetxt("dut_dconv.txt", o_dut)
np.savetxt("dur_dconv.txt", o_dur)


assert o_dut.shape == o_dur.shape, f"shape test failed. { o_dut.shape} != {o_dur.shape}"
assert np.all(o_dut == o_dur), f"val test failed. Check dut_conv.txt and dur_conv.txt files "

print("test passed!")

ValueError: operands could not be broadcast together with shapes (3,3,3) (3,3,16) 

# create numpy model

In [ ]:
import json
from NumpyModel import Model
from Layers import InputLayer, Conv2D, Dense, DephtwiseConv2D, Flatten, Softmax


classifier_model_path = "/home/george/Desktop/dnn_processor/reference_algorithm/neural_network/saved_models/classifier_v2/"

modelJson = json.load(open(classifier_model_path + "model.json"))

model = Model()


def create_activation(activation_name):
    if activation_name == "softmax":
        return Softmax()
    elif activation_name == "tanh":
        return Tanh()
    elif activation_name == "linear":
        return None
    else:
        raise TypeError(f"unknow activation name {activation_name}")
    return None


for layerConfig in modelJson["layers"]:
    if layerConfig["layerType"] == "InputLayer":
        layer = InputLayer(layerConfig["layerName"])

    elif layerConfig["layerType"] == "Conv2D":
        layer = Conv2D(
            padding=layerConfig["layer_config"]["padding"],
            strides=layerConfig["layer_config"]["strides"],
            name=layerConfig["layerName"],
        )
        layer.load_weights(
            classifier_model_path + layerConfig["weightPath"],
            classifier_model_path + layerConfig["biasPath"],
        )

        layer.activation = create_activation(layerConfig["activation"])

    elif layerConfig["layerType"] == "DepthwiseConv2D":
        layer = DephtwiseConv2D(layerConfig["layerName"])
        layer.load_weights(
            classifier_model_path + layerConfig["weightPath"],
            classifier_model_path + layerConfig["biasPath"],
        )
        layer.activation = create_activation(layerConfig["activation"])

    elif layerConfig["layerType"] == "Dense":
        layer = Dense(layerConfig["layerName"])
        layer.load_weights(
            classifier_model_path + layerConfig["weightPath"],
            classifier_model_path + layerConfig["biasPath"],
        )
        layer.activation = create_activation(layerConfig["activation"])

    elif layerConfig["layerType"] == "Flatten":
        layer = Flatten(layerConfig["layerName"])

    elif layerConfig["layerType"] == "Dropout":
        print("ignoring not implemented dropout")
        continue

    else:
        print(layerConfig)
        raise NotImplementedError(layerConfig["layerName"])

    model.layers.append(layer)

print(model.layers)